In [1]:
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from sklearn.model_selection import train_test_split

sys.path.append("..")

from utils import paths

pd.set_option("display.max_columns", None)

tf.config.list_physical_devices()


2023-06-29 01:22:58.358571: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-29 01:22:58.384550: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 01:22:58.824777: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-29 01:22:59.691554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
data = pd.read_csv(paths.DATA_PATH + "/COVID19MX_sample.csv")
data.sample(10)


,SEXO,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,DEFUNCION,INTUBADO,NEUMONIA,EDAD,EMBARAZO,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,CLASIFICACION_FINAL,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
26037,1,2,2022-02-05,2022-02-05,1,2.0,2.0,82.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,2.0
33460,2,2,2022-01-16,2022-01-11,1,2.0,2.0,80.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,2.0
93297,2,2,2022-01-20,2022-01-10,1,2.0,2.0,95.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,2.0
12804,2,1,2022-06-19,2022-06-19,0,3.0,2.0,76.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,3.0
77211,2,2,2022-01-31,2022-01-29,1,2.0,2.0,95.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,7,109.0,0.0,2.0
2647,2,2,2022-01-15,2022-01-14,1,2.0,2.0,69.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,109.0,0.0,2.0
89798,1,1,2022-12-27,2022-12-25,0,3.0,2.0,38.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7,109.0,0.0,3.0
17067,2,1,2022-01-24,2022-01-15,0,3.0,2.0,49.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,7,109.0,0.0,3.0
62522,2,1,2022-07-22,2022-07-17,0,3.0,2.0,29.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3,109.0,0.0,3.0
42864,2,2,2022-12-01,2022-12-01,1,1.0,2.0,61.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,NaN,7,109.0,0.0,2.0


In [3]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93458 entries, 0 to 93457
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   SEXO                 93458 non-null  int64  
 1   TIPO_PACIENTE        93458 non-null  int64  
 2   FECHA_INGRESO        93458 non-null  object 
 3   FECHA_SINTOMAS       93458 non-null  object 
 4   DEFUNCION            93458 non-null  int64  
 5   INTUBADO             93186 non-null  float64
 6   NEUMONIA             92920 non-null  float64
 7   EDAD                 93269 non-null  float64
 8   EMBARAZO             93147 non-null  float64
 9   DIABETES             93123 non-null  float64
 10  EPOC                 93137 non-null  float64
 11  ASMA                 93154 non-null  float64
 12  INMUSUPR             93152 non-null  float64
 13  HIPERTENSION         93155 non-null  float64
 14  OTRA_COM             92299 non-null  float64
 15  CARDIOVASCULAR       93149 non-null 

In [4]:
X = data.drop(columns=["DEFUNCION", "FECHA_INGRESO", "FECHA_SINTOMAS"]).to_numpy()
Y = data[["DEFUNCION"]].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=557)

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")


Train: (74766, 21)
Test: (18692, 21)


In [5]:
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=2, num_classes=2,
)
clf.fit(
    x=X_train, y=Y_train,
    epochs=50, validation_split=0.25
)

2023-06-29 01:22:59.837470: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-29 01:22:59.837601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-29 01:22:59.837655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
True              |True              |structured_data_block_1/normalize
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
False             |False             |structured_data_block_1/dense_block_1/use_batchnorm
0                 |0                 |structured_data_block_1/dense_block_1/dropout
32                |32                |structured_data_block_1/dense_block_1/units_0
32                |32                |structured_data_block_1/dense_block_1/units_1
0                 |0                 |classification_head_1/dropout
adam              |adam              |optimizer
0.001             |0.001             |learning_rate



2023-06-29 01:23:02.247288: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype int64 and shape [74766,1]
	 [[{{node Placeholder/_20}}]]
2023-06-29 01:23:02.247460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [74766,21]
	 [[{{node Placeholder/_0}}]]
2023-06-29 01:23:03.837284: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and sha

KeyboardInterrupt: 